In [14]:
import pandas as pd
import json
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re

import requests
import json
from tqdm import tqdm
from emoji import demojize
from nltk.tokenize import TweetTokenizer
import os
from datetime import datetime
import sys
import os
sys.path.append("../src")
import prompt_utils
from reproduce_model import normalizeTweet

import numpy as np

import pandas as pd
from collections import Counter

# Define a list of filenames to load

for k in range(0, 5):
    filename = f"../data/labeled_data/generic_test_{k}.json"
    with open(filename) as f:
        data = json.load(f)
    df_train = pd.DataFrame(data["train"])
    df_test = pd.DataFrame(data["test"])
    df_valid = pd.DataFrame(data["valid"])

    augmented_df = df_train.copy()
    for idx, row in df_train.iterrows():
        tweet = normalizeTweet(row.text)
        prompt, _, request_params = prompt_utils.get_vicuna_data_augmentation(tweet, prompt_utils.get_base_request_params())
        response = prompt_utils.get_response(request_params, prompt, "")
        """print(tweet)
        print("-------------")
        print(row.annotations)
        print("-------------")
        print(prompt)
        print("-------------")
        print(response)
        print("-------------")"""
        augmented_df = augmented_df.append({"text": tweet, "annotations": row.annotations}, ignore_index=True)   
        if idx % 640 == 0:
            print(idx % 640)

    augmented_df.to_csv(f"../data/labeled_data/generic_test_{k}_augmented_vicuna_v01.csv", index=False)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [16]:
len(augmented_df)

6400

In [26]:

import ast

for k in range(0, 5):
    filename = f"../data/labeled_data/augmented_test_{k}.csv"
    df_train = pd.read_csv(filename)

    df_train["annotations"] = df_train["annotations"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

    train_annotations = df_train["annotations"].tolist()
    # Get all unique classes
    global classes
    classes = set()
    for annotation in train_annotations:
        classes.update(annotation)
    classes = sorted(list(classes))
    print("Classes: ", classes)


    for idx, row in df_train.iterrows():
        # less than symbol: <
        # greater than symbol: >
        # ampersand: & 
        if idx < 3201:
            df_train.loc[idx, "text"] = normalizeTweet(row["text"])
            df_train.loc[idx, "annotations"] = ast.literal_eval(row["annotations"])
        if idx >= 3201:
            print(row)
            df_train.loc[idx, "annotations"] = ast.literal_eval(row["annotations"])

Classes:  ['Conspiracy Theory', 'Education', 'Election Campaign', 'Environment', 'Government/Public', 'Health', 'Immigration/Integration', 'Justice/Crime', 'Labor/Employment', 'Macroeconomics/Economic Regulation', 'Media/Journalism', 'Others', 'Religion', 'Science/Technology', 'War/Terror']


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:25                                                                                   │
│                                                                                                  │
│   22 │   │   # ampersand: &                                                                      │
│   23 │   │   if idx < 3201:                                                                      │
│   24 │   │   │   df_train.loc[idx, "text"] = normalizeTweet(row["text"])                         │
│ ❱ 25 │   │   │   df_train.loc[idx, "annotations"] = ast.literal_eval(row["annotations"])         │
│   26 │   │   if idx >= 3201:                                                                     │
│   27 │   │   │   print(row)                                                                      │
│   28 │   │   │   df_train.loc[idx, "annotations"] = ast.literal_eval(row["annotations"])         │
│                                                                                                  │
│ /media/bruno/0d2f61d2-2b9c-4043-9a46-8e4dfe74fc95/bruno/anaconda3/envs/my_env/lib/python3.11/ast │
│ .py:110 in literal_eval                                                                          │
│                                                                                                  │
│    107 │   │   │   │   else:                                                                     │
│    108 │   │   │   │   │   return left - right                                                   │
│    109 │   │   return _convert_signed_num(node)                                                  │
│ ❱  110 │   return _convert(node_or_string)                                                       │
│    111                                                                                           │
│    112                                                                                           │
│    113 def dump(node, annotate_fields=True, include_attributes=False, *, indent=None):           │
│                                                                                                  │
│ /media/bruno/0d2f61d2-2b9c-4043-9a46-8e4dfe74fc95/bruno/anaconda3/envs/my_env/lib/python3.11/ast │
│ .py:109 in _convert                                                                              │
│                                                                                                  │
│    106 │   │   │   │   │   return left + right                                                   │
│    107 │   │   │   │   else:                                                                     │
│    108 │   │   │   │   │   return left - right                                                   │
│ ❱  109 │   │   return _convert_signed_num(node)                                                  │
│    110 │   return _convert(node_or_string)                                                       │
│    111                                                                                           │
│    112                                                                                           │
│                                                                                                  │
│ /media/bruno/0d2f61d2-2b9c-4043-9a46-8e4dfe74fc95/bruno/anaconda3/envs/my_env/lib/python3.11/ast │
│ .py:83 in _convert_signed_num                                                                    │
│                                                                                                  │
│     80 │   │   │   │   return + operand                                                          │
│     81 │   │   │   else:                                                                         │
│     82 │   │   │   │   return - operand                                                          │
│ ❱   83 │   │   return _convert_num(node)                   

In [13]:
len(augmented_df)

3203

In [2]:
prompt_utils.get_vicuna_data_augmentation("awdawd", prompt_utils.get_base_request_params())

('### Human:\nPlease data augment the following Tweet.\n\nTweet: awdawd\n\n### Assistant:\nAugmented Tweet: ',
 '',
 {'prompt': None,
  'context': None,
  'max_new_tokens': 500,
  'do_sample': True,
  'temperature': 0.7,
  'top_p': 0.1,
  'typical_p': 1,
  'repetition_penalty': 1.2,
  'encoder_repetition_penalty': 1.0,
  'top_k': 40,
  'min_length': 0,
  'no_repeat_ngram_size': 0,
  'num_beams': 1,
  'penalty_alpha': 0,
  'length_penalty': 1,
  'early_stopping': False,
  'seed': -1,
  'stopping_strings': ['\n\n', '### Human:', '### Assistant:']})